In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch
from tensor_fusion.fusion_model import AdaptiveRankFusion_with_AdaptiveRankSubNets
from tensor_fusion.dataset import get_cmu_mosi_dataset

/home/christian_lee/anaconda3/envs/tensor_layers/lib/python3.8/site-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


In [2]:
device='cuda'
DTYPE=torch.float64
train_set, valid_set, test_set = get_cmu_mosi_dataset(binary=True, device=device, dtype=DTYPE)

In [7]:
input_dims = (5, 20, 300)
hidden_dims = (4, 16, 128)
text_out = 64
dropouts = (0.3, 0.3, 0.3, 0.3)
rank = 4
output_dim = 1
model = AdaptiveRankFusion_with_AdaptiveRankSubNets(input_dims, hidden_dims, dropouts, output_size=1, max_rank=20, prior_type='half_cauchy', eta=0.01, device='cuda', dtype=DTYPE)

In [4]:
batch_size=32
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=len(valid_set))
test_dataloader = DataLoader(test_set, batch_size=len(test_set))

In [5]:
def get_log_prior_coeff(epoch, warm_up_epochs, no_log_prior_epochs):

    coeff = (epoch - no_log_prior_epochs) / warm_up_epochs
    return min(max(coeff, 0), 1)

In [8]:
train_losses = []
valid_losses = []
ranks = {'fusion': [], 'audio_1': [], 'audio_2': [], 'audio_3': [],
         'video_1': [], 'video_2': [], 'video_3': [],
         'text_1': [], 'text_2': []}
warm_up_epochs = 50
no_log_prior_epochs = 5
criterion = nn.BCEWithLogitsLoss(reduction='sum')
factor_lr = 0.0005
lr = 0.001
subnet_params = list(model.audio_subnet.parameters()) + list(model.video_subnet.parameters()) + list(model.text_subnet.parameters())
optimizer = optim.Adam([{"params": subnet_params, "lr": lr}, 
                        {"params": list(model.fusion_layer.parameters()), "lr": factor_lr}])
#opimizer = optim.Adam(list(model.parameters()), lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, verbose=True)
epochs = 100
for e in range(epochs):
    print('Epoch {}'.format(e))
    train_loss = 0.0
    model.train()
    #print(model.fusion_layer.weight_tensor.rank_parameter)
    ranks['fusion'].append(model.fusion_layer.weight_tensor.estimate_rank())
    ranks['audio_1'].append(model.audio_subnet.linear_1.weight_tensor.estimate_rank())
    ranks['audio_2'].append(model.audio_subnet.linear_2.weight_tensor.estimate_rank())
    ranks['audio_3'].append(model.audio_subnet.linear_3.weight_tensor.estimate_rank())
    ranks['video_1'].append(model.video_subnet.linear_1.weight_tensor.estimate_rank())
    ranks['video_2'].append(model.video_subnet.linear_2.weight_tensor.estimate_rank())
    ranks['video_3'].append(model.video_subnet.linear_3.weight_tensor.estimate_rank())
    ranks['text_1'].append(model.text_subnet.rnn.layer_ih.weight_tensor.estimate_rank())
    ranks['text_2'].append(model.text_subnet.rnn.layer_hh.weight_tensor.estimate_rank())
    
    for text, audio, vision, label in train_dataloader:
        model.zero_grad()
        output = model(audio, vision, text)
        loss = criterion(output, label) - 0.0001 * get_log_prior_coeff(e, warm_up_epochs, no_log_prior_epochs) * model.get_log_prior()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    train_loss = train_loss / len(train_set)
    print(train_loss)
    train_losses.append(train_loss)
    model.eval()
    for text, audio, vision, label in valid_dataloader:
        output = model(audio, vision, text)
        valid_loss = criterion(output, label).item()
    valid_loss = valid_loss / len(valid_set)
    print(valid_loss)
    valid_losses.append(valid_loss)
    scheduler.step(valid_loss)

Epoch 0
0.6953655111132397
0.6906828014704174
Epoch 1
0.6870417331401912
0.686652832571739
Epoch 2
0.6787156716119
0.6772143925250744
Epoch 3
0.6580133082639696
0.6697519400325476
Epoch 4
0.6499958951542446
0.6628715763503222
Epoch 5
0.6440059321963829
0.6540843229787077
Epoch 6
0.6353020175256148
0.6474120451391987
Epoch 7
0.6238769391392395
0.6354410431205696
Epoch 8
0.6182358690019231
0.6362192456055691
Epoch 9
0.5942593223749684
0.6240671961357569
Epoch 10
0.6094387361187752
0.6237292011169038
Epoch 11
0.5799613244380164
0.6038424430758238
Epoch 12
0.5783630735950096
0.6021947944186168
Epoch 13
0.5760933948067962
0.6044161090076376
Epoch 14
0.5440340918440989
0.5897001113325132
Epoch 15
0.5425904569166256
0.5841852121826332
Epoch 16
0.5598793890023002
0.5859850106651009
Epoch 17
0.5267581652274311
0.5843660335224752
Epoch 18
0.5330992748338205
0.57113593405361
Epoch 19
0.5164586978814727
0.5794577954235834
Epoch 20
0.5200590800191018
0.5774355648229456
Epoch 21
0.5071496227708334
0

In [6]:
batch_size = 32
epochs = 50
# settings from https://github.com/Justin1904/TensorFusionNetworks/blob/master/train.py
input_dims = (5, 20, 300)
hidden_dims = (4, 16, 128)
text_out = 64
dropouts = (0.3, 0.3, 0.3, 0.3)
post_fusion_dim = 32
model = TFN(input_dims, hidden_dims, text_out, dropouts, post_fusion_dim, device=device, dtype=DTYPE)

In [7]:
epochs = 30
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(list(model.parameters())[2:])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)
for e in range(epochs):
    print('Epoch {}'.format(e))
    train_loss = 0.0
    model.train()
    for text, audio, vision, label in train_dataloader:
        model.zero_grad()
        output = model(audio, vision, text)
        loss = criterion(output, label)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(train_loss)
    
    model.eval()
    for text, audio, vision, label in valid_dataloader:
        output = model(audio, vision, text)
        valid_loss = criterion(output, label).item()
    scheduler.step(valid_loss)
    print(valid_loss)

Epoch 0
27.746791183948517
0.5983000993728638
Epoch 1
22.599841356277466
0.5741891860961914
Epoch 2
20.9776431620121
0.5182520747184753
Epoch 3
18.447107419371605
0.5316178798675537
Epoch 4
16.16508023440838
0.5781940817832947
Epoch 5
13.750149548053741
0.5866302847862244
Epoch 6
12.41133339703083
Epoch     7: reducing learning rate of group 0 to 1.0000e-04.
0.5868079662322998
Epoch 7
9.790406368672848
0.6064910292625427
Epoch 8
9.155158430337906
0.6138084530830383
Epoch 9
8.670896269381046
0.6321585178375244
Epoch 10
8.77745607495308
Epoch    11: reducing learning rate of group 0 to 1.0000e-05.
0.6368790864944458
Epoch 11
8.624453119933605
0.6399129629135132
Epoch 12
8.29000923037529
0.6426395177841187
Epoch 13
8.115225300192833
0.6466211080551147
Epoch 14
7.943665310740471
Epoch    15: reducing learning rate of group 0 to 1.0000e-06.
0.64751797914505
Epoch 15
7.860414206981659
0.6442150473594666
Epoch 16
8.334999352693558
0.6477405428886414
Epoch 17
7.960166782140732
0.64896446466445

In [7]:
batch_size = 32
epochs = 50
# settings from https://github.com/Justin1904/TensorFusionNetworks/blob/master/train.py
input_dims = (5, 20, 300)
hidden_dims = (4, 16, 128)
text_out = 64
dropouts = (0.3, 0.3, 0.3, 0.3)
rank = 4
output_dim = 1
model = LMF(input_dims, hidden_dims, text_out, dropouts, output_dim, rank, use_softmax=False, device=device, dtype=DTYPE)

In [8]:
criterion = nn.BCEWithLogitsLoss()
factors = list(model.parameters())[:3]
other = list(model.parameters())[3:]
factor_lr = 0.0005
lr = 0.001
optimizer = optim.Adam([{"params": factors, "lr": factor_lr}, {"params": other, "lr": lr}])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, verbose=True)
for e in range(epochs):
    print('Epoch {}'.format(e))
    train_loss = 0.0
    model.train()
    for text, audio, vision, label in train_dataloader:
        model.zero_grad()
        output = model(audio, vision, text)
        loss = criterion(output, label)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(train_loss)
    
    model.eval()
    for text, audio, vision, label in valid_dataloader:
        output = model(audio, vision, text)
        valid_loss = criterion(output, label).item()
    scheduler.step(valid_loss)
    print(valid_loss)

Epoch 0
27.707884550094604
0.6103724837303162
Epoch 1
25.751307010650635
0.5993456244468689
Epoch 2
21.946386218070984
0.506530225276947
Epoch 3
19.738655865192413
0.5119958519935608
Epoch 4
17.642503082752228
0.4785168170928955
Epoch 5
16.091687828302383
0.5398954153060913
Epoch 6
14.24790771305561
0.5091980695724487
Epoch 7
11.752081230282784
0.49786096811294556
Epoch 8
10.936970323324203
0.6509965658187866
Epoch 9
8.470754906535149
0.7462157011032104
Epoch 10
6.700329817831516
Epoch    11: reducing learning rate of group 0 to 5.0000e-05.
Epoch    11: reducing learning rate of group 1 to 1.0000e-04.
0.6205970048904419
Epoch 11
4.487218387424946
0.656859278678894
Epoch 12
4.011319886893034
0.7138518691062927
Epoch 13
2.7622850690968335
0.7538882493972778
Epoch 14
2.608662152197212
0.8387965559959412
Epoch 15
2.2656643863301724
0.8688085675239563
Epoch 16
1.9815785735845566
Epoch    17: reducing learning rate of group 0 to 5.0000e-06.
Epoch    17: reducing learning rate of group 1 to 1

In [11]:
model.fusion_layer.weight_tensor.factors[0].dtype

torch.float32

In [27]:
list(model.fusion_layer.rank_param)

[tensor(0.0361, grad_fn=<UnbindBackward>),
 tensor(0.4688, grad_fn=<UnbindBackward>),
 tensor(0.1688, grad_fn=<UnbindBackward>),
 tensor(0.7413, grad_fn=<UnbindBackward>),
 tensor(0.5441, grad_fn=<UnbindBackward>),
 tensor(0.8754, grad_fn=<UnbindBackward>),
 tensor(0.3675, grad_fn=<UnbindBackward>),
 tensor(0.9385, grad_fn=<UnbindBackward>),
 tensor(0.8079, grad_fn=<UnbindBackward>),
 tensor(0.3892, grad_fn=<UnbindBackward>)]

In [16]:
model.fusion_layer.rank_param

Parameter containing:
tensor([0.0560, 0.3947, 0.4498, 0.4022, 0.3426, 0.5294, 0.5075, 0.6149, 0.6144,
        0.5896], requires_grad=True)